In [ ]:
import numpy as np
import pandas as pd

# Load the dataset
df = pd.read_csv('/home/gourango/Downloads/Datasets/diabetes.csv')
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
y = np.where(y == 0, -1, 1)  # Convert labels to -1 and 1

# Split data manually
def train_test_split_manual(X, y, test_size=0.2, random_state=42):
    np.random.seed(random_state)
    indices = np.random.permutation(len(X))
    test_size = int(len(X) * test_size)
    test_indices, train_indices = indices[:test_size], indices[test_size:]
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

X_train, X_test, y_train, y_test = train_test_split_manual(X, y)

# Standardize the dataset
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

# Implement an improved SVM using gradient descent with momentum
class SVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=2000, momentum=0.9):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.momentum = momentum
        self.w = None
        self.b = None
        self.velocity_w = None
        self.velocity_b = None
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0
        self.velocity_w = np.zeros(n_features)
        self.velocity_b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y[idx] * (np.dot(x_i, self.w) + self.b) >= 1
                if condition:
                    grad_w = 2 * self.lambda_param * self.w
                    grad_b = 0
                else:
                    grad_w = 2 * self.lambda_param * self.w - np.dot(x_i, y[idx])
                    grad_b = -y[idx]
                
                self.velocity_w = self.momentum * self.velocity_w - self.lr * grad_w
                self.velocity_b = self.momentum * self.velocity_b - self.lr * grad_b
                
                self.w += self.velocity_w
                self.b += self.velocity_b
    
    def predict(self, X):
        approx = np.dot(X, self.w) + self.b
        return np.sign(approx)

# Train the SVM model
svm = SVM(n_iters=3000)
svm.fit(X_train, y_train)

# Make predictions
y_pred = svm.predict(X_test)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f'Accuracy: {accuracy:.2f}')

# Function to predict user input
def predict_class():
    print("\nEnter the following values for prediction:")
    try:
        user_input = [float(input(f"{col}: ")) for col in df.columns[:-1]]
        user_data = np.array(user_input).reshape(1, -1)
        user_data = (user_data - mean) / std  # Scale input
        user_pred = svm.predict(user_data)
        print("\nPredicted Outcome:", "Diabetic" if user_pred[0] == 1 else "Non-Diabetic")
    except ValueError:
        print("Invalid input! Please enter numeric values for all features.")

predict_class()


Accuracy: 0.76

Enter the following values for prediction:
